# Imports



In [1]:
!pip install pulp

     |████████████████████████████████| 14.2 MB 25.8 MB/s 


In [2]:
import itertools
from pulp import *

# Brute force


In [ ]:
a1 = 8000
a2 = 11000
a3 = 6000
a4 = 4000
tab_a = [a1, a2, a3, a4]
m1 = 6700
m2 = 10000
m3 = 5500
m4 = 3400
liste_projets = [a1, a2, a3, a4]
critere = 19000

In [ ]:
 combinaisons = [list(itertools.combinations(liste_projets, i)) for i in range(len(liste_projets))]
 combinaisons

[[()],
 [(8000,), (11000,), (6000,), (4000,)],
 [(8000, 11000),
  (8000, 6000),
  (8000, 4000),
  (11000, 6000),
  (11000, 4000),
  (6000, 4000)],
 [(8000, 11000, 6000),
  (8000, 11000, 4000),
  (8000, 6000, 4000),
  (11000, 6000, 4000)]]

In [ ]:
def modif_combinaisons(combinaisons):
  res = []
  for i in range(len(combinaisons)):
    for j in range(len(combinaisons[i])):
      if (list(combinaisons[i][j]) != []):
        res.append(list(combinaisons[i][j]))
  return res

combinaisons = modif_combinaisons(combinaisons)
print(combinaisons)

[[8000], [11000], [6000], [4000], [8000, 11000], [8000, 6000], [8000, 4000], [11000, 6000], [11000, 4000], [6000, 4000], [8000, 11000, 6000], [8000, 11000, 4000], [8000, 6000, 4000], [11000, 6000, 4000]]


In [ ]:
def somme_critère(tab_une_combinaison, a1, a2, a3, a4, m1, m2, m3, m4):
  somme = 0
  for i in range(len(tab_une_combinaison)):
    if (tab_une_combinaison[i] == a1):
      somme += m1
    elif (tab_une_combinaison[i] == a2):
      somme += m2
    elif (tab_une_combinaison[i] == a3):
      somme += m3
    elif (tab_une_combinaison[i] == a4):
      somme += m4
  return(somme)

In [ ]:
sommes_criteres = []
somme_combinaisons = []
for i in range(len(combinaisons)):
  sommes_criteres.append(somme_critère(combinaisons[i], a1, a2, a3, a4, m1, m2, m3, m4))
  somme_combinaisons.append(sum(combinaisons[i]))

print("Somme des critères : ", sommes_criteres)
print("Somme des combinaisons : ", somme_combinaisons)

Somme des critères :  [6700, 10000, 5500, 3400, 16700, 12200, 10100, 15500, 13400, 8900, 22200, 20100, 15600, 18900]
Somme des combinaisons :  [8000, 11000, 6000, 4000, 19000, 14000, 12000, 17000, 15000, 10000, 25000, 23000, 18000, 21000]


In [ ]:
def resultat_maximisation(sommes_criteres, sommes_combinaisons, critere, combinaisons):
  res_somme_combinaisons = sommes_combinaisons[0]
  res_somme_criteres = sommes_criteres[0]
  meilleure_combinaison = []
  for i in range(len(sommes_criteres)):
    if (sommes_criteres[i] <= critere):
      if(res_somme_combinaisons < sommes_combinaisons[i]):
        res_somme_combinaisons = sommes_combinaisons[i]
        res_somme_criteres = sommes_criteres[i]
        meilleure_combinaison = combinaisons[i]
  retour = [res_somme_combinaisons, res_somme_criteres, meilleure_combinaison]
  return(retour)


res_maxim_brute_force = resultat_maximisation(sommes_criteres, somme_combinaisons, critere, combinaisons)
print("Meilleure somme combinaison : ", res_maxim_brute_force[0])
print("Meilleure somme critères : ", res_maxim_brute_force[1])
print("Meilleure combinaison : ", res_maxim_brute_force[2])

Meilleure somme combinaison :  21000
Meilleure somme critères :  18900
Meilleure combinaison :  [11000, 6000, 4000]


In [ ]:
def association_combinaison_valeur(meilleure_combinaison, tab_a):
  chaine = "";
  for i in range(len(tab_a)):
    if (i > 0):
      chaine += " ; "
    if (tab_a[i] in meilleure_combinaison):
      chaine += "x" + str(i+1) + " = 1"
    else:
      chaine += "x" + str(i+1) + " = 0"
  print(chaine)

association_combinaison_valeur(res_maxim_brute_force[2], tab_a)

x1 = 0 ; x2 = 1 ; x3 = 1 ; x4 = 1


# Avec le module pulp

Avec la seule contrainte du budget

In [ ]:
#Créer le problème
prob = LpProblem("sac_a_dos", LpMaximize)

#Créer les variables
x1 = LpVariable("x1", lowBound=0, cat="Binary")
x2 = LpVariable("x2", lowBound=0, cat="Binary")
x3 = LpVariable("x3", lowBound=0, cat="Binary")
x4 = LpVariable("x4", lowBound=0, cat="Binary")

#La fonction à optimiser

prob += 8000*x1+11000*x2+6000*x3+4000*x4

#Les contraintes

prob += 6700*x1+10000*x2+5500*x3+3400*x4 <= 19000

#La résolution

sol = prob.solve()

#Affichage de la solution

print("x1 = ", value(x1), " ; x2 = ", value(x2), " ; x3 = ", value(x3), " ; x4 = ", value(x4), "\nRésultat = ", value(prob.objective))

x1 =  0.0  ; x2 =  1.0  ; x3 =  1.0  ; x4 =  1.0 
Résultat =  21000.0


Avec la condition deux investissements max autorisés

In [3]:
#Créer le problème
prob = LpProblem("sac_a_dos", LpMaximize)

#Créer les variables
x1 = LpVariable("x1", lowBound=0, cat="Binary")
x2 = LpVariable("x2", lowBound=0, cat="Binary")
x3 = LpVariable("x3", lowBound=0, cat="Binary")
x4 = LpVariable("x4", lowBound=0, cat="Binary")

#La fonction à optimiser

prob += 8000*x1+11000*x2+6000*x3+4000*x4

#Les contraintes

prob += 6700*x1+10000*x2+5500*x3+3400*x4 <= 19000
prob += x1+x2+x3+x4 <= 2

#La résolution

sol = prob.solve()

#Affichage de la solution

print("x1 = ", value(x1), " ; x2 = ", value(x2), " ; x3 = ", value(x3), " ; x4 = ", value(x4), "\nRésultat = ", value(prob.objective))

x1 =  1.0  ; x2 =  1.0  ; x3 =  0.0  ; x4 =  0.0 
Résultat =  19000.0


Avec les contraintes : "si x2 alors x4" et "x1 et x3 ne peuvent pas être faits ensembles"

In [7]:
#Créer le problème
prob = LpProblem("sac_a_dos", LpMaximize)

#Créer les variables
x1 = LpVariable("x1", lowBound=0, cat="Binary")
x2 = LpVariable("x2", lowBound=0, cat="Binary")
x3 = LpVariable("x3", lowBound=0, cat="Binary")
x4 = LpVariable("x4", lowBound=0, cat="Binary")

#La fonction à optimiser

prob += 8000*x1+11000*x2+6000*x3+4000*x4

#Les contraintes

prob += 6700*x1+10000*x2+5500*x3+3400*x4 <= 19000
prob += x2-x4 <= 0
prob += x1+x3 <= 1

#La résolution

sol = prob.solve()

#Affichage de la solution

print("x1 = ", value(x1), " ; x2 = ", value(x2), " ; x3 = ", value(x3), " ; x4 = ", value(x4), "\nRésultat = ", value(prob.objective))

x1 =  0.0  ; x2 =  1.0  ; x3 =  1.0  ; x4 =  1.0 
Résultat =  21000.0
